### Importing the required packages


In [1]:
using JetReconstruction
using LorentzVectorHEP
import LorentzVectorHEP.eta
using Plots
using StatsBase 
using DelimitedFiles
using LoopVectorization

### Implementing functions to load data from a file

In [2]:
# function to read the full data file 
function readFile(file::Int64)
    datafile = open("./total/$(file).txt","r")
    lines = readlines(datafile)
    close(datafile)

    lines
end;

# function to create PseudoJets from a set of data
# taking in px, py, pz, E of the particle
function loadParticles(lines::Array{String})
    jetData = Vector{PseudoJet}(undef, length(lines))

    i = 1
    for line in lines
        
        arr = split(line, "\t")           
        px = parse(Float64, arr[1])
        py = parse(Float64, arr[2])
        pz = parse(Float64, arr[3])
        E = parse(Float64, arr[4])
        
        jetData[i] = PseudoJet(px, py, pz, E)
        i += 1
    end

    jetData
end;

# function for jet reconstruction
# uses Cambridge-Aachen algorithm with jet radius 1.0 by default
function makeJets(data::Vector{PseudoJet}, mtd = 0, rad = 1.0)
    finaljets = jet_reconstruct(data; p = mtd, R = rad, recombine = +, strategy = RecoStrategy.Best)
    finaljets
end;



### Implementing histogram plotting function


In [3]:
function plotHIST(jets::Vector{PseudoJet}, choice::Int)
    NUM = length(jets)
    values = []
    
    if (choice === 1)
        for j in 1:NUM
            pt = pt2(jets[j]) ^ 0.5
            push!(values, pt)
        end

    elseif(choice === 2)
        for j in 1:NUM
            pseuRap = eta(jets[j])
            # pseuRap = jets[j]._rap
            # push!(values, pseuRap)
            -100 < pseuRap < 100 ? push!(values, pseuRap) : continue
        end
        
    elseif(choice === 3)
        for j in 1:NUM
            PHI = phi(jets[j])
            push!(values, PHI)
        end

    else
        println("Enter a valid choice.")
        return
    end


    MIN = minimum(values)
    MAX = maximum(values)

    x = LinRange(MIN, MAX+1, 51)

    println("Mean = $(mean(values))")
    println("Standard deviation = $(std(values))")
    display(histogram(values, bins=x))#, yaxis=(:log10)))
    # savefig(histogram(pt, bins=x, yaxis=(:log10)), "pt-julia.png")
    # sleep(10)
end;

### Defining some utility functions

In [4]:
m2(p) = JetReconstruction.m2(p);
pt2(p) = JetReconstruction.pt2(p);
eta(p) = JetReconstruction.eta(p);
phi(p) = JetReconstruction.phi(p);
NonexistentParent = JetReconstruction.NonexistentParent;
BeamJet = JetReconstruction.BeamJet;

# function to see if  a jet has parent jets
function has_parents(jet, history)
    
    N = jet._cluster_hist_index
    p1 = history[N].parent1
    p2 = history[N].parent2
    # println(N)
    p1 === p2 === NonexistentParent ? result = false : result = true
    return result, p1, p2

end;

# Function to calculate ΔR
function delta_R(jet1, jet2)
    eta1, phi1 = jet1._rap, jet1._phi
    eta2, phi2 = jet2._rap, jet2._phi

    d_eta = eta1 - eta2
    d_phi = phi1 - phi2
    d_phi = abs(d_phi) > π ? 2π - abs(d_phi) : d_phi
    return sqrt(d_eta^2 + d_phi^2)
end;

# Function to calculate kt distance between two pseudojets
function kt_distance(jet1::PseudoJet, jet2::PseudoJet, R = 1)
    p1 = pt2(jet1)
    p2 = pt2(jet2)
    
    d_R = delta_R(jet1, jet2)
    return min(p1, p2) * (d_R^2 / R^2)
end;

# Function to get the constituents of a particular jet
function jet_constituents(jet::PseudoJet, clusterseq::ClusterSequence)
    constituents = Vector{PseudoJet}(undef, 0)
    
    hist = clusterseq.history
    allJets = clusterseq.jets

    had_parents, p1, p2 = has_parents(jet, hist)
    
    if (had_parents)
        parent1 = allJets[hist[p1].jetp_index]
        parent2 = allJets[hist[p2].jetp_index]

        append!(constituents,jet_constituents(parent1, clusterseq))
        append!(constituents,jet_constituents(parent2, clusterseq))
    else 
        push!(constituents, jet)
        return constituents
    end 

end;

# Function to recluster the constituents of a jet in a different method
function recluster(jet::PseudoJet, clusterseq::ClusterSequence, rad = 1.0, mtd = 0)
    constituents = jet_constituents(jet, clusterseq)
    new_clusterseq = jet_reconstruct(constituents; p = mtd, R = rad, strategy = RecoStrategy.N2Plain)
   
    return new_clusterseq
end;

function sort_jets!(event_jet_array)
    jet_pt(jet) = pt2(jet)^0.5
    sort!(event_jet_array, by = jet_pt, rev = true)
    # println(event_jet_array)
end;

function join(jets::Vector{PseudoJet})
    px = py = pz = E = 0.0

    for jet in jets
        px += jet.px
        py += jet.py
        pz += jet.pz
        E += jet.E
    end

    combinedJet = PseudoJet(px, py, pz, E)
    combinedJet
end;

function get_inclusive_jets(clusterseq::ClusterSequence, ptmin = 0.0)
    jets = Vector{PseudoJet}(undef, 0)
    dcut = ptmin * ptmin

    for elt in clusterseq.history
        elt.parent2 == BeamJet || continue;
        iparent_jet = clusterseq.history[elt.parent1].jetp_index;
        jet = clusterseq.jets[iparent_jet];
    
        if (pt2(jet) >= dcut)
            push!(jets, jet);
        end;
    
    end;

    jets
end;


### Implementing the Mass Drop Tagger

In [5]:
struct MassDropTagger
    mu::Float64
    y::Float64
end;

function apply_massdrop(jet::PseudoJet, clusterseq::ClusterSequence, tag::MassDropTagger)
    allJets = clusterseq.jets
    hist = clusterseq.history

    while(true)
        had_parents, p1, p2 = has_parents(jet, hist)

        if (had_parents)
            parent1 = allJets[hist[p1].jetp_index]
            parent2 = allJets[hist[p2].jetp_index]

            if (m2(parent1) < m2(parent2))
                p1, p2 = p2, p1
                parent1, parent2 = parent2, parent1
            end

            if ((m2(parent1) < m2(jet)*tag.mu^2) && (kt_distance(parent1, parent2) > tag.y*m2(jet)))
                return jet
            else
                jet = parent1
            end
    
        else 
            return PseudoJet(0.0, 0.0, 0.0, 0.0)
        end

    end

end;


In [13]:
fullData = readFile(0)
particles = loadParticles(fullData)

@time data = makeJets(particles, 0, 1.0)

count = 1

ptmin = 2.0

allJets = get_inclusive_jets(data, ptmin)
taggedJets = Vector{PseudoJet}(undef, 0)
println(length(allJets))

for jet in allJets
    global count
    mdt = MassDropTagger(0.67, 0.09)
    @time trial = apply_massdrop(jet, data, mdt)
    push!(taggedJets, trial)
    println(count, "\tOriginal jet: pt = $(pt2(jet) ^ 0.5), eta = $(eta(jet)), phi = $(phi(jet)), E = $(jet.E)")
    println(count, "\tTagged jet: pt = $(pt2(trial) ^ 0.5), eta = $(eta(trial)), phi = $(phi(trial)), E = $(trial.E)")
    println()

    count += 1
end

mdt = MassDropTagger(0.67, 0.09)
filtered = [apply_massdrop(jet, data, mdt) for jet in allJets]
sort_jets!(filtered)

println(allJets)
open("./results/massdrop-julia.csv", "w") do FILE
    write(FILE, "pt,eta,m\n")

    for jet in filtered
        if pt2(jet) > 0
            println(pt2(jet)^0.5,"\t",sign(m2(jet))*abs(m2(jet))^0.5)
            write(FILE, "$(pt2(jet)^0.5),$(eta(jet)),$(sign(m2(jet))*abs(m2(jet))^0.5)\n")
        end
    end

end


  0.000131 seconds (777 allocations: 115.469 KiB)
11
  0.000007 seconds (1 allocation: 80 bytes)
1	Original jet: pt = 26.130848232425834, eta = 3.406422621141889, phi = 2.6749116992524002, E = 394.5340289934504
1	Tagged jet: pt = 0.0, eta = 0.0, phi = 0.0, E = 0.0

  0.000001 seconds (1 allocation: 80 bytes)
2	Original jet: pt = 2.125720970693987, eta = 2.418204608888067, phi = 2.6716158036886775, E = 12.163646946977643
2	Tagged jet: pt = 0.0, eta = 0.0, phi = 0.0, E = 0.0

  0.000001 seconds (1 allocation: 80 bytes)
3	Original jet: pt = 2.0086398616206367, eta = -1.5974788148568575, phi = 5.05065699790494, E = 5.620447377513666
3	Tagged jet: pt = 0.0, eta = 0.0, phi = 0.0, E = 0.0

  0.000001 seconds
4	Original jet: pt = 4.373967011936128, eta = -0.245263138384402, phi = 1.46563570138571, E = 5.153599395423116
4	Tagged jet: pt = 4.373967011936128, eta = -0.245263138384402, phi = 1.46563570138571, E = 5.153599395423116

  0.000001 seconds
5	Original jet: pt = 12.586904469090056, eta = 

### Implementing the Soft Drop Tagger

In [8]:
struct SoftDropTagger
    zcut::Float64
    b::Float64
end;

function apply_softdrop(jet::PseudoJet, clusterseq::ClusterSequence, rad::Float64, tag::SoftDropTagger)
    new_clusterseq = recluster(jet, clusterseq, rad, 0)
    new_jet = get_inclusive_jets(new_clusterseq)[1]

    allJets = new_clusterseq.jets
    hist = new_clusterseq.history
    
    while(true)
        had_parents, p1, p2 = has_parents(new_jet, hist)

        if (had_parents)
            parent1 = allJets[hist[p1].jetp_index]
            parent2 = allJets[hist[p2].jetp_index]

            pti = pt2(parent1)^0.5
            ptj = pt2(parent2)^0.5

            if (m2(parent1) < m2(parent2))
                p1, p2 = p2, p1
                parent1, parent2 = parent2, parent1
            end

            if (min(pti, ptj)/(pti + ptj) > tag.zcut * (delta_R(parent1, parent2)/rad) ^ tag.b)
                return new_jet
            else
                new_jet = parent1
            end
    
        else 
            return PseudoJet(0.0, 0.0, 0.0, 0.0)
        end

    end

end;


In [10]:
fullData = readFile(0)
particles = loadParticles(fullData)

@time data = makeJets(particles, -1, 1.0)

mdt = SoftDropTagger(0.1, 2.0)
count = 1

ptmin = 2.0

allJets = get_inclusive_jets(data, ptmin)
length(allJets)
taggedJets = Vector{PseudoJet}(undef, 0)
println(length(allJets))

for jet in allJets
    global count
    @time trial = apply_softdrop(jet, data, 1.0, mdt)
    push!(taggedJets, trial)
    println(count, "\tOriginal jet: pt = $(pt2(jet) ^ 0.5), eta = $(eta(jet)), phi = $(phi(jet)), E = $(jet.E)")
    println(count, "\tTagged jet: pt = $(pt2(trial) ^ 0.5), eta = $(eta(trial)), phi = $(phi(trial)), E = $(trial.E)")
    println()

    count += 1
end

filtered = [apply_softdrop(jet, data, 1.0, mdt) for jet in allJets]
sort_jets!(filtered)

open("./results/softdrop-julia.csv", "w") do FILE
    write(FILE, "pt,eta,m\n")
    for jet in filtered

        if pt2(jet) > 0
            final = jet
            # println(pt2(final)^0.5,"\t",sign(m2(final))*abs(m2(final))^0.5)
            write(FILE, "$(pt2(final)^0.5),$(eta(final)),$(sign(m2(final))*abs(m2(final))^0.5)\n")
        end
    end
end


  0.000190 seconds (773 allocations: 115.203 KiB)
9
  0.147255 seconds (130.85 k allocations: 6.247 MiB, 99.95% compilation time)
1	Original jet: pt = 26.130848232425834, eta = 3.406422621141889, phi = 2.6749116992524007, E = 394.5340289934503
1	Tagged jet: pt = 25.230948732943812, eta = 3.379827053371096, phi = 2.6857282124998707, E = 370.95059202783114

  0.000027 seconds (211 allocations: 32.516 KiB)
2	Original jet: pt = 25.156074535747507, eta = -2.2511071775624996, phi = 5.259380832613704, E = 121.29322998338885
2	Tagged jet: pt = 23.189650402038197, eta = -2.2643806315908557, phi = 5.286708126323866, E = 113.05344713237298

  0.000006 seconds (85 allocations: 9.109 KiB)
3	Original jet: pt = 12.680797211997875, eta = 1.7778892659226257, phi = 6.136677084752503, E = 39.54085507853854
3	Tagged jet: pt = 12.680797211997872, eta = 1.7778892659226257, phi = 6.136677084752503, E = 39.54085507853854

  0.000008 seconds (66 allocations: 6.594 KiB)
4	Original jet: pt = 5.084848379504678, e

### Implementing Jet Filtering

In [11]:
struct Filter
    filterRadius::Float64 
    numHardestJets::Int
end;

function apply_filter(jet::PseudoJet, clusterseq::ClusterSequence, filter::Filter)
    rad = filter.filterRadius;
    new_clusterseq = recluster(jet, clusterseq, rad, 0)
    reclustered = sort_jets!(get_inclusive_jets(new_clusterseq))

    n = length(reclustered) <= filter.numHardestJets ? length(reclustered) : filter.numHardestJets 
    hard = reclustered[1:n]
    
    filtered = join(hard)
    
    filtered    
end;


### Using the Filter

In [14]:
F = Filter(0.3, 3);
filtered = [apply_filter(jet, data, F) for jet in allJets]
sort_jets!(filtered)

open("./results/filter-julia.csv", "w") do FILE
    write(FILE, "pt,eta,m\n")

    for jet in filtered
        final = jet
        println(pt2(final)^0.5,"\t",sign(m2(final))*abs(m2(final))^0.5)
        write(FILE, "$(pt2(final)^0.5),$(eta(final)),$(sign(m2(final))*abs(m2(final))^0.5)\n")
    end
end



25.425973541479426	7.210829212029251
19.38426838643532	4.940888635018613
12.019482608698311	6.671551849243314
4.118477623409108	2.3690413802846306
4.070582559375079	2.3285876831086623
2.487682330918309	1.3796583711669919
2.1864737162176358	2.1299068355434225
2.125720970693987	1.8245962501869402
1.986731603795744	2.234241203271326
1.7682801418035814	1.742592726752077
1.6337885215517716	0.8683760196849447


### Implementing Jet Trimming

In [15]:
struct Trim
    trimRadius::Float64
    trimFraction::Float64
    reclusterMethod::Int
end;

function apply_trim(jet::PseudoJet, clusterseq::ClusterSequence, trim::Trim)
    constituents = jet_constituents(jet, clusterseq);
    rad = trim.trimRadius;
    mtd = trim.reclusterMethod
    frac2 = trim.trimFraction ^ 2

    reclustered = get_inclusive_jets(plain_jet_reconstruct(constituents; p = mtd, R = rad));
    sort_jets!(reclustered);
    
    hard = Vector{PseudoJet}(undef, 0)
    for item in reclustered
        if pt2(item) >= frac2 * pt2(jet)
            push!(hard, item)
        end
    end
    trimmed = join(hard)
    
    trimmed    
end;

In [16]:
T = Trim(0.3, 0.3, 0);

filtered = [apply_trim(jet, data, T) for jet in allJets]
sort_jets!(filtered)

open("./results/trim-julia.csv", "w") do FILE
    write(FILE, "pt,eta,m\n")

    for jet in filtered
        if (pt2(jet) > 0)
            final = jet
            println(pt2(final)^0.5,"\t",sign(m2(final))*abs(m2(final))^0.5)
            write(FILE, "$(pt2(final)^0.5),$(eta(final)),$(sign(m2(final))*abs(m2(final))^0.5)\n")
        end
    end
end



24.0024657948891	5.931458819456554
11.39058332934317	1.3116812665798905
8.958194272123187	2.466264379925911
3.5649614011641453	1.385375128391125
2.1376462300800347	1.172218172070111
2.125720970693987	1.8245962501869402
1.8254297278011031	0.9395700000000008
1.6925532105156107	2.0683933326506128
1.539181615169957	0.935056154256617


### Implementing JH Top Tagging

In [6]:
struct JHTopTagger
    delP::Float64
    delR::Float64
    cosTW::Float64
end

function manhattan_dist(jet1::PseudoJet, jet2::PseudoJet)
    y1 = JetReconstruction.rapidity(jet1)
    y2 = JetReconstruction.rapidity(jet2)
    ϕ1 = phi(jet1)
    ϕ2 = phi(jet2)

    return abs(y1 - y2) + abs(ϕ1 - ϕ2)
end

function JH_decluster(jet::PseudoJet, clusterseq::ClusterSequence, Δp::Float64, Δr::Float64, pt0::Float64)

    allJets = clusterseq.jets
    hist = clusterseq.history

    
    new_jet = jet
    
    while(true)
        jets = Vector{PseudoJet}(undef, 0)
        had_parents, p1, p2 = has_parents(new_jet, hist)

        if (had_parents)
            global jets
            parent1 = allJets[hist[p1].jetp_index]
            parent2 = allJets[hist[p2].jetp_index]

            pti = pt2(parent1)^0.5
            ptj = pt2(parent2)^0.5

            if ((pti/pt0 < Δp && ptj/pt0 < Δp) || (manhattan_dist(parent1, parent2) < Δr))
                push!(jets, PseudoJet(0.0, 0.0, 0.0, 0.0))
                break

            elseif ((pti < ptj) && (pti/pt0 < Δp))
                new_jet = parent2

            elseif ((ptj < pti) && (ptj/pt0 < Δp))
                new_jet = parent1

            else 
                push!(jets, parent1)
                push!(jets, parent2)
                break
            end
    
        else 
            push!(jets, PseudoJet(0.0, 0.0, 0.0, 0.0))
            break
        end

    end

    if length(jets) === 1
        push!(jets, jet)
    end

    return jets

end


function apply_JHTagger(jet::PseudoJet, clusterseq::ClusterSequence, tagger::JHTopTagger)
    Δp = tagger.delP
    Δr = tagger.delR
    cosΘW = tagger.cosTW

    allJets = clusterseq.jets
    hist = clusterseq.history
    jet_pt = pt2(jet)^0.5

    new_jet = jet
    
    while(true)
        had_parents, p1, p2 = has_parents(new_jet, hist)

        if (had_parents)
            parent1 = allJets[hist[p1].jetp_index]
            parent2 = allJets[hist[p2].jetp_index]

            pti = pt2(parent1)^0.5
            ptj = pt2(parent2)^0.5

            if ((pti/jet_pt < Δp && ptj/jet_pt < Δp) || (manhattan_dist(parent1, parent2) < Δr))
                # return PseudoJet(0.0, 0.0, 0.0, 0.0)
                return [PseudoJet(0.0, 0.0, 0.0, 0.0), []]


            elseif ((pti < ptj) && (pti/jet_pt < Δp))
                new_jet = parent2

            elseif ((ptj < pti) && (ptj/jet_pt < Δp))
                new_jet = parent1

            else 
                arr1 = JH_decluster(parent1, clusterseq, Δp, Δr, jet_pt)
                arr2 = JH_decluster(parent2, clusterseq, Δp, Δr, jet_pt)

                final = vcat(arr1, arr2)

                filter!(e -> (e.px ≠ 0.0 && e.py ≠ 0.0 && e.pz ≠ 0.0), final)
                if length(final) > 2
                    # return join(final)
                    return [join(final), final]
                else
                    # return PseudoJet(0.0, 0.0, 0.0, 0.0)
                    return [PseudoJet(0.0, 0.0, 0.0, 0.0), []]
                end

            end
    
        else 
            # return PseudoJet(0.0, 0.0, 0.0, 0.0)
            return [PseudoJet(0.0, 0.0, 0.0, 0.0), []]
        end

    end

end


apply_JHTagger (generic function with 1 method)

### Using JH Top Tagger

In [8]:
fullData = readFile(0)
particles = loadParticles(fullData)

@time data = makeJets(particles, 0, 1.0)
# println(a)

jht = JHTopTagger(0.1, 0.19, 0.7)
count = 1

ptmin = 2.0

allJets = get_inclusive_jets(data, ptmin)
taggedJets = Vector{PseudoJet}(undef, 0)
allConstituents = []

for jet in allJets
    global count
    trial, cons = apply_JHTagger(jet, data, jht);
    push!(taggedJets, trial)
    push!(allConstituents, cons)
    println(count, "\tOriginal jet: pt = $(pt2(jet) ^ 0.5), eta = $(eta(jet)), phi = $(phi(jet)), E = $(jet.E)")
    println(count, "\tTagged jet: pt = $(pt2(trial) ^ 0.5), eta = $(eta(trial)), phi = $(phi(trial)), E = $(trial.E)")
    println()

    count += 1
end

sort_jets!(taggedJets)

open("./results/jhtop-julia.csv", "w") do FILE
    write(FILE, "pt,eta,m\n")
    for jet in taggedJets

        if pt2(jet) > 0
            final = jet
            # println(pt2(final)^0.5,"\t",sign(m2(final))*abs(m2(final))^0.5)
            write(FILE, "$(pt2(final)^0.5),$(eta(final)),$(sign(m2(final))*abs(m2(final))^0.5)\n")
        end
    end
end

  0.000150 seconds (777 allocations: 115.469 KiB)
1	Original jet: pt = 26.130848232425834, eta = 3.406422621141889, phi = 2.6749116992524002, E = 394.5340289934504
1	Tagged jet: pt = 0.0, eta = 0.0, phi = 0.0, E = 0.0

2	Original jet: pt = 2.125720970693987, eta = 2.418204608888067, phi = 2.6716158036886775, E = 12.163646946977643
2	Tagged jet: pt = 0.0, eta = 0.0, phi = 0.0, E = 0.0

3	Original jet: pt = 2.0086398616206367, eta = -1.5974788148568575, phi = 5.05065699790494, E = 5.620447377513666
3	Tagged jet: pt = 2.008639861620637, eta = -1.5974788148568575, phi = 5.05065699790494, E = 5.620447377513666

4	Original jet: pt = 4.373967011936128, eta = -0.245263138384402, phi = 1.46563570138571, E = 5.153599395423116
4	Tagged jet: pt = 4.373967011936127, eta = -0.24526313838440209, phi = 1.46563570138571, E = 5.153599395423116

5	Original jet: pt = 12.586904469090056, eta = 1.7809156033724232, phi = 6.129839442886363, E = 39.341040840487125
5	Tagged jet: pt = 12.019482608698311, eta = 1

### Benchmarking the different modules

In [21]:
trials = 15

fileName = ["events-ee-Z.hepmc3", "events-ee-H.hepmc3", "events-pp-0.5TeV-5GeV.hepmc3", "events-pp-1TeV-5GeV.hepmc3", "events-pp-2TeV-5GeV.hepmc3", "events-pp-1TeV.hepmc3", "events-pp-2TeV.hepmc3", "events-pp-5TeV-10GeV.hepmc3", "events-pp-8TeV-20GeV.hepmc3", "events-pp-13TeV-20GeV.hepmc", "events-pp-20TeV-20GeV.hepmc3", "events-pp-20TeV-50GeV.hepmc3", "events-pp-30TeV-50GeV.hepmc3"];

open("./results/time-jhtop-julia.csv", "w") do FILE
    write(FILE, "FileName,mean_particles,n_samples,time_per_event\n")
    for eventFILE in fileName
        allEvents = read_final_state_particles("./benchmarks/data/$eventFILE")
        avgTime = 0.0

        for j in 1:trials
            TIME = 0.0
            global numCounts = 0
            for i in 1:100
                numCounts += length(allEvents[i])

                cluster = makeJets(allEvents[i], 1.0)
                allJets = get_inclusive_jets(cluster, 2.0)
                
                # mdt = MassDropTagger(0.67, 0.09)
                # TIME += @elapsed (tagged = [apply_massdrop(jet, cluster, mdt) for jet in allJets])
                
                # sdt = SoftDropTagger(0.1, 2.0)
                # TIME += @elapsed (tagged = [apply_softdrop(jet, cluster, sdt) for jet in allJets])

                # filter = Filter(0.3, 3)
                # TIME += @elapsed (filtered = [apply_filter(jet, cluster, filter) for jet in allJets])

                # trim = Trim(0.3, 0.3, 0)
                # TIME += @elapsed (trimmed = [apply_trim(jet, cluster, trim) for jet in allJets])
                
                F = JHTopTagger(0.1, 0.19, 0.7);
                TIME += @elapsed (filtered = [apply_JHTagger(jet, cluster, F) for jet in allJets])
            
            end
            avgTime += TIME

            numCounts /= 100 
        end

        avgTime /= trials

        println("$eventFILE\t$(numCounts)\t$(trials)\t$(avgTime*10^4)\n")
        write(FILE, "$eventFILE,$(numCounts),$(trials),$(avgTime*10^4)\n")
        # print(i, "\t", avgTime)

    end
end

[ Info: Total Events: 100


events-ee-Z.hepmc3	43.05	15	10.114247999999993

events-ee-H.hepmc3	64.97	15	2.157544666666667



[ Info: Total Events: 100


events-pp-0.5TeV-5GeV.hepmc3	112.62	15	5.840753333333335



[ Info: Total Events: 100


...

events-pp-20TeV-20GeV.hepmc3	524.59	15	24.47176066666667



[ Info: Total Events: 100


events-pp-20TeV-50GeV.hepmc3	553.64	15	22.439842666666667



[ Info: Total Events: 100


events-pp-30TeV-50GeV.hepmc3	632.29	15	25.783572

